In [ ]:
-- Create database
CREATE DATABASE OrderManagement;

In [1]:
-- Create tables
-- Customers Table
CREATE TABLE customers (
    customer_id SERIAL PRIMARY KEY,
    customer_name VARCHAR(255) NOT NULL,
    phone VARCHAR(255),
    address_line_1 VARCHAR(255),
    address_line_2 VARCHAR(255),
    city VARCHAR(50),
    state VARCHAR(50),
    postal_code VARCHAR(50),
    country VARCHAR(50),
    territory VARCHAR(50),
    contact_last_name VARCHAR(100),
    contact_first_name VARCHAR(100)
);

-- Products Table
CREATE TABLE products (
    product_code VARCHAR(50) PRIMARY KEY,
    product_line VARCHAR(255),
    msrp DECIMAL(10, 2)
);

-- Orders Table
CREATE TABLE orders (
    order_number INT PRIMARY KEY,
    order_date DATE,
    status VARCHAR(50),
    qtr_id INT,
    month_id INT,
    year_id INT,
    deal_size VARCHAR(50),
    customer_id INT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);

-- OrderDetails Table
CREATE TABLE order_details (
    order_detail_id SERIAL PRIMARY KEY,
    order_number INT,
    product_code VARCHAR(50),
    quantity_ordered INT,
    price_each DECIMAL(10, 2),
    order_line_number INT,
    sales DECIMAL(10, 2),
    FOREIGN KEY (order_number) REFERENCES orders(order_number),
    FOREIGN KEY (product_code) REFERENCES products(product_code)
);

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.021

In [1]:
import pandas as pd
# Read CSV file 
df = pd.read_csv('/Users/meganwagnersmith/Desktop/OrderManagementData/sales_data_sample.csv', encoding='ISO-8859-1')
print(df.columns)

Index(['order_number', 'quantity_ordered', 'price_each', 'order_line_number',
       'sales', 'order_date', 'status', 'qtr_id', 'month_id', 'year_id',
       'product_line', 'msrp', 'product_code', 'customer_name', 'phone',
       'address_line_1', 'address_line_2', 'city', 'state', 'postal_code',
       'country', 'territory', 'contact_last_name', 'contact_first_name',
       'deal_size'],
      dtype='object')


In [2]:
# Extracting unique customers
customers_df = df[['customer_name', 'contact_first_name', 'contact_last_name', 'phone', 'address_line_1', 'address_line_2', 'city', 'state', 'postal_code', 'country', 'territory']].drop_duplicates('customer_name')
customers_df['customer_id'] = range(1, 1+len(customers_df))
# customers_df['contact_full_name'] = customers_df['contact_first_name'] + ' ' + customers_df['contact_last_name']
customers_df = customers_df[['customer_id','customer_name', 'contact_first_name', 'contact_last_name', 'phone', 'address_line_1', 'address_line_2', 'city', 'state', 'postal_code', 'country', 'territory']]

# Extracting orders
orders_df = df[['order_number', 'order_date', 'status', 'customer_name', 'qtr_id', 'month_id', 'year_id', 'deal_size']].drop_duplicates('order_number')
orders_df = orders_df.merge(customers_df[['customer_name', 'customer_id']], on='customer_name', how='left')
orders_df = orders_df[['order_number', 'order_date', 'status', 'customer_id', 'qtr_id', 'month_id', 'year_id', 'deal_size']]

# Extracting order details
order_details_df = df[['order_number', 'product_code', 'quantity_ordered', 'price_each', 'order_line_number', 'sales']].copy()
order_details_df['order_detail_id'] = range(1, 1+len(order_details_df))
order_details_df = order_details_df[['order_detail_id', 'order_number', 'product_code', 'quantity_ordered', 'price_each', 'order_line_number', 'sales']].copy()

# Extracting products 
products_df = df[['product_code', 'product_line', 'msrp']].drop_duplicates()

print(customers_df.head())
print(orders_df.head())
print(order_details_df.head())
print(products_df.head())

   customer_id             customer_name contact_first_name contact_last_name  \
0            1         Land of Toys Inc.               Kwai                Yu   
1            2        Reims Collectables               Paul           Henriot   
2            3           Lyon Souveniers             Daniel          Da Cunha   
3            4         Toys4GrownUps.com              Julie             Young   
4            5  Corporate Gift Ideas Co.              Julie             Brown   

              phone                 address_line_1 address_line_2  \
0        2125557818        897 Long Airport Avenue            NaN   
1        26.47.1555             59 rue de l'Abbaye            NaN   
2  +33 1 46 62 7555  27 rue du Colonel Pierre Avia            NaN   
3        6265557265             78934 Hillside Dr.            NaN   
4        6505551386                7734 Strong St.            NaN   

            city state postal_code country territory  
0            NYC    NY       10022     USA 

In [3]:
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine

load_dotenv()
API_KEY = os.getenv("API_KEY")

# Replace these values with your database credentials
username = os.getenv('MY_USERNAME')
password = os.getenv('MY_PASSWORD')
host = 'localhost'
port = '5432'
database = 'ordermanagement'

engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')

In [4]:
# Insert data into customers table
customers_df.to_sql('customers', engine, if_exists='append', index=False)


# Insert data into products table
products_df.to_sql('products', engine, if_exists='append', index=False)

# Insert data into orders table
orders_df.to_sql('orders', engine, if_exists='append', index=False)

# Insert data into order details table
order_details_df.to_sql('order_details', engine, if_exists='append', index=False)



823

In [1]:
-- View total sales by customer
SELECT 
    c.customer_name, 
    SUM(od.sales) as total_sales
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_details od ON o.order_number = od.order_number
GROUP BY c.customer_name
ORDER BY total_sales DESC;

(92 row(s) affected)

Total execution time: 00:00:00.011

customer_name,total_sales
Euro Shopping Channel,912294.11
Mini Gifts Distributors Ltd.,654858.06
"Australian Collectors, Co.",200995.41
Muscle Machine Inc,197736.94
La Rochelle Gifts,180124.90
"Dragon Souveniers, Ltd.",172989.68
Land of Toys Inc.,164069.44
The Sharp Gifts Warehouse,160010.27
"AV Stores, Co.",157807.81
"Anna's Decorations, Ltd",153996.13


In [8]:
-- Top 10 most ordered products
SELECT 
    p.product_code,
    SUM(od.quantity_ordered) as total_quantity_ordered
FROM products p
JOIN order_details od ON p.product_code = od.product_code
GROUP BY p.product_code
ORDER BY total_quantity_ordered DESC
LIMIT 10; 

(10 row(s) affected)

Total execution time: 00:00:00.024

product_code,total_quantity_ordered
S18_3232,1774
S24_3856,1052
S18_4600,1031
S700_4002,1029
S12_4473,1024
S24_3949,1008
S18_1097,999
S50_1341,999
S18_2432,998
S18_3856,997


In [9]:
-- Count by order status
SELECT 
    status,
    COUNT(order_number) as order_count
FROM orders
GROUP BY status
ORDER BY order_count DESC;

(6 row(s) affected)

Total execution time: 00:00:00.004

status,order_count
Shipped,286
In Process,6
Resolved,4
Cancelled,4
On Hold,4
Disputed,3


In [14]:
-- Avg Order Value by Customer
SELECT 
    c.customer_name,
    ROUND(AVG(od.sales), 2) as avg_order_value
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_details od on o.order_number = od.order_number
GROUP BY c.customer_name
ORDER BY avg_order_value DESC;


(92 row(s) affected)

Total execution time: 00:00:00.010

customer_name,avg_order_value
Super Scale Inc.,4674.83
Mini Caravy,4233.60
"La Corne D'abondance, Co.",4226.25
Royale Belge,4180.01
Muscle Machine Inc,4119.52
Gift Depot Inc.,4075.79
"UK Collectables, Ltd.",4069.25
Danish Wholesale Imports,4028.93
"Dragon Souveniers, Ltd.",4023.02
The Sharp Gifts Warehouse,4000.26
